<div>
    <table>
        <tr>
            <td>
                <center>
                    <h1>Wurst Introduction</h1>
                     <a href="https://www.psi.ch/en/ta/people/romain-sacchi">Karin Treyer</a> (PSI)
                    <br><br>
                    Duration: 1 hour 30 minutes.
                </center>
            </td>
        </tr>
    </div>

Github repository: [https://github.com/polca/wurst](https://github.com/polca/wurst)

Documentation: [https://wurst.readthedocs.io/](https://wurst.readthedocs.io/)

Wurst authors: Chris Mutel, Brian Cox

<div class="alert alert-info">
Note: we will be using <a href="https://docs.brightway.dev/en/latest/content/installation/index.html">Brightway 2.5</a>, not <a href="https://docs.brightway.dev/en/legacy/">Brightway 2</a>. From the user end side, very little differ between the two. The code executed throughout this notebook works with both versions... probably.
</div>


### Scenario: A world without fossil fuels

#### Background:
By 2050, fossil fuels in power generation are phased out due to stringent climate policies and technological advancements in renewables.


#### Objective:

1. Replace the contribution of fossil-based technologies in electricity markets with non-hydro renewables throughout the database.
The former share of fossil-based electricity should be redistributed evenly across already supplying renewables.

2. Compare the electricity markets, at low-voltage level with respect to Climate Change, Fine Particle emissions and Abiotic Resource Depletion. Can you identify some trade-offs?

### 1. Import:

Load the necessary libraries (`wurst`, `bw2data`, and maybe a plotting library like `matplotlib`).

In [41]:
import wurst
import wurst.searching as ws
import bw2io, bw2data
import matplotlib.pyplot as plt
from prettytable import PrettyTable

Select/activate a `bw2data` project where you have an ecoinvent database.

In [3]:
list(bw2data.projects)

[Project: default,
 Project: ei39,
 Project: wurst,
 Project: ei38,
 Project: new4,
 Project: ei38_2,
 Project: ei382,
 Project: ei310,
 Project: ecoinvent-3.10-EN15804,
 Project: ecoinvent-3.10-consequential,
 Project: ecoinvent-3.10-cutoff,
 Project: ecoinvent-3.9.1-cutoff,
 Project: ecoinvent-3.7.1-cutoff]

In [4]:
bw2data.projects.set_current("ecoinvent-3.10-cutoff")

Using `wurst`, extract the ecoinvent database to turn it into a list of dictionaries.

In [52]:
db = wurst.extract_brightway2_databases("ecoinvent-3.10-cutoff")

Getting activity data


100%|█████████████████████████████████| 23523/23523 [00:00<00:00, 424548.88it/s]


Adding exchange data to activities


100%|████████████████████████████████| 743409/743409 [00:18<00:00, 39988.91it/s]


Filling out exchange data


100%|██████████████████████████████████| 23523/23523 [00:00<00:00, 29091.13it/s]


Build `wurst` filters to select only markets for electricity (high, medium, and low voltage). Filter out datasets specific to industries (coal, aluminium, railways, etc.) or for certified renewable electricity.

In [53]:
market_filters = [
    ws.contains("name", "market for electricity"),
    ws.equals("unit", "kilowatt hour"),
    ws.exclude(ws.either(
        *[
            ws.contains("name", name) for name in [
                "product",
                "generic",
                "aluminium",
                "reuse",
                "industry",
                "Railways",
                "mining"
            ]
        ]
    )),
]

Iterate through the database using the filters you built above. Inside each dataset, iterate over the technosphere exchanges. You may need to ignore exchanges that represent market losses, for example. Try to distinguish suppliers of fossil-based electricity from renewable ones.

In [54]:
fossil_suppliers = [
    "coal", "gas", "lignite", "oil", "peat"
]
renewable_suppliers = [
    "photovoltaic", "wind"
]

In [55]:
technosphere_filters = [
    ws.equals("unit", "kilowatt hour"),
    ws.exclude(ws.contains("name", "market for")),
    ws.exclude(ws.contains("name", "voltage")),
]

In [56]:
modifications = []

for ds in ws.get_many(
    db,
    *market_filters
):
    # first, we check if there are renewable electricity suppliers
    number_renewables = len([x for x in ws.technosphere(ds, *technosphere_filters) if any(y in x["name"] for y in renewable_suppliers)])
    # we also need to check if there are fossil electricity suppliers
    number_fossil = len([x for x in ws.technosphere(ds, *technosphere_filters) if any(y in x["name"] for y in fossil_suppliers)])

    if number_fossil > 0 and number_renewables > 0:
        shares = {}
        # we iterate a first time to collect current shares
        for techno in ws.technosphere(
            ds,
            *technosphere_filters
            ):
            shares[(techno["name"], techno["location"])] = {"old": techno["amount"]}
    
        # we iterate a second time to zero-out fossil-based electricity
        surplus = 0
        for techno in ws.technosphere(
            ds,
            *technosphere_filters
            ):
            
            if any(x in techno["name"] for x in fossil_suppliers):
                modifications.append(
                    [ds["name"][:30], ds["location"], techno["name"][:30], techno["amount"], 0]
                )
                surplus += techno["amount"]
                techno["amount"] = 0
    
        # we iterate a third time to distribute the surplus
        
        for techno in ws.technosphere(
            ds,
            *technosphere_filters
            ):
            if any(x in techno["name"] for x in renewable_suppliers):
                new_share = shares[(techno["name"], techno["location"])]["old"] + (surplus / number_renewables)
                modifications.append(
                    [ds["name"][:30], ds["location"], techno["name"][:30], techno["amount"], new_share]
                )
                techno["amount"] = new_share

table = PrettyTable(["Name", "Loc", "Tech", "Old share", "New share",])
table.add_rows(modifications)
print(table)

+--------------------------------+-----------------------+--------------------------------+-----------------------+-----------------------+
|              Name              |          Loc          |              Tech              |       Old share       |       New share       |
+--------------------------------+-----------------------+--------------------------------+-----------------------+-----------------------+
| market for electricity, high v |           PT          | heat and power co-generation,  |   0.0036260679984655  |           0           |
| market for electricity, high v |           PT          |  electricity production, oil   |   0.0142027727897208  |           0           |
| market for electricity, high v |           PT          | electricity production, hard c |   0.0443906388980669  |           0           |
| market for electricity, high v |           PT          | electricity production, natura |    0.23758306351299   |           0           |
| market for electri